In [ ]:
import ezomero as ez
from omero.model import WellI
from getpass import getpass

In [ ]:
def combine_plates(conn, source_plates, target_plate, sort_run_names=True, safety_ownership=True, safety_screen=True):
    """
    Move the plate acquisitions of the source plate to the target
    plate. When wells are missing from the target plate, new wells
    are created in the target plate. The order in which the plates are merged matters, 
    and is affected by the sort_run_names parameter.
    parameters:
      - conn: connection object to OMERO
      - source_plates. iterable of plate IDs to merge into the target plate
      - target_plate: the ID of the plate into which the source_plates are merged
      - sort_run_names: process the runs of the given plates in alphabetical order
      - safety_ownership: Safety to ensure that the plates processed are owned by the current user
      - safety_screen: Safety to ensure that the plates processed are all linked to the same screen
    """
    update_service = conn.getUpdateService()
    
    if type(source_plates) is int:
        source_plates = [source_plates]
    
    source_plates = list(conn.getObjects("Plate", source_plates))
    target_plate_o = conn.getObject("Plate", target_plate)

    screen_ids = set()
    for plate_o in source_plates + [target_plate_o]:
        if safety_ownership:
            assert plate_o.getOwner().getId() == conn.getUserId(), f"'Safety ownership' error: The plate {plate_o.getId()} is not owned by the user"
        if safety_screen:
            screen_o = plate_o.getParent()
            assert screen_o is not None, f"'Safety screen' error: the plate {plate.getId()} is not part of a screen"
            screen_ids.add(screen_o.getId())
    if safety_screen:
        assert len(screen_ids) == 1, f"Safety 'screen' error: plates belong to different screens, {screen_ids}"
    
    target_d = {}
    for well in target_plate_o.listChildren():
        target_d[well.getWellPos()] = well._obj
    # Populating target with missing wells
    for plate in source_plates:
        for well in plate.listChildren():
            if well.getWellPos() not in target_d.keys():
                new_well = WellI()
                new_well.setColumn(well._obj.getColumn())
                new_well.setRow(well._obj.getRow())
                new_well.setPlate(target_plate_o._obj)
                target_d[well.getWellPos()] = update_service.saveAndReturnObject(new_well)
                print(f"Create {well.getWellPos()}")

    # Sort of all run names from all plates
    plate_run_l = []
    for source_plate_o in source_plates:
        plate_run_l.extend([(source_plate_o, run_o) for run_o in source_plate_o.listPlateAcquisitions()])
    if sort_run_names:
        plate_run_l = sorted(plate_run_l, key=lambda x: x[1].getName())
        
    for source_plate_o, run_o in plate_run_l:  
        for well in source_plate_o.listChildren():
            well_oi = target_d[well.getWellPos()]
            
            for ws in filter(lambda x: x._obj.plateAcquisition._id._val == run_o.getId(),
                             well.listChildren()):
                ws._obj.setWell(well_oi)
                well_oi.addWellSample(ws._obj)

        for key, well_oi in target_d.items():
            _ = update_service.saveAndReturnObject(well_oi)

        run_o._obj.setPlate(target_plate_o._obj)
        _ = update_service.saveAndReturnObject(run_o._obj)
        
        # Need to reload all target objects here
        target_plate_o = conn.getObject("Plate", target_plate)
        target_d = {}
        for well in target_plate_o.listChildren():
            target_d[well.getWellPos()] = well._obj


In [ ]:
GROUP = ""
USER = ""
PWD = getpass("password:")
HOST = ""
PORT = 4064

In [ ]:
with ez.connect(USER, host=HOST, port=PORT, secure=True, group=GROUP) as conn:
    combine_plates(conn, source_plates=[2078,2079], target_plate=2076, sort_run_names=True, safety_ownership=True, safety_screen=True)